# Cvičení 6 - Hřebenová regrese, bias-variance tradeoff

V tomto notebooku se budeme zabývat: 
* hřebenovou regresí 
* analýzou vztahu vychýlení a rozptylu
* základními statistickými vlastnostmi odhadu

In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

np.set_printoptions(precision=5, suppress=True)  # suppress scientific float notation (so 0.000 is printed as 0.)

## Načtení dat

Využijeme opět data o cenách domů v oblasti Bostonu v USA ([více info zde](https://www.kaggle.com/c/boston-housing)), která máme tentokrát v mírně modifikované variantě uložená v souboru `data.csv`.

Data jsou již vyčištěná. Proměnná, kterou chceme predikovat je `medv`.

In [ ]:
df = pd.read_csv('data.csv').astype('float64')
print('Shape', df.shape)
df.head()

### Příprava trénovací a testovací množiny

Využijeme [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) ze `scikit-learn`.

In [ ]:
random_seed = 42

Xtrain, Xval, ytrain, yval = train_test_split(df.drop(columns = ['medv']), df['medv'], test_size=0.4, random_state=random_seed)
Xval, Xtest, yval, ytest = train_test_split(Xval, yval, test_size=0.3, random_state=random_seed)

print(f"Train rozměry, X: {Xtrain.shape}, y: {ytrain.shape}")
print(f"Val rozměry, X: {Xval.shape}, y: {yval.shape}")
print(f"Test rozměry, X: {Xtest.shape}, y: {ytest.shape}")

#### Vytvoříme model lineární regrese pro porovnání

In [ ]:
clf_lr = LinearRegression()
clf_lr.fit(Xtrain, ytrain)

print(f"w_hat_OLS: {clf_lr.intercept_:.4f}", clf_lr.coef_)

RMSE_val_lr = mean_squared_error(yval, clf_lr.predict(Xval), squared = False)
print(f'Validační RMSE: {RMSE_val_lr:.5f}')

RMSE_test_lr = mean_squared_error(ytest, clf_lr.predict(Xtest), squared = False)
print(f'Testovací RMSE: {RMSE_test_lr:.5f}')

plt.scatter(yval, clf_lr.predict(Xval))
plt.plot([0,50], [0,50], 'r')
plt.show()

## Úkol  - implementujte ručně model hřebenové regrese

Podobně jako v minulém cvičení využijte pouze funkce z `numpy`.

Do předpřipravené třídy `my_ridge` implementujte metody fit a predict, tak aby:
* metoda `fit` vyrobila odhad $\hat{\boldsymbol w}_{\lambda} = (\mathbf{X}^T \mathbf X + \lambda \mathbf{I}')^{-1} \mathbf X^T \boldsymbol Y$, uložila ho do proměnné `self.w_hat`.
* metoda `predict` spočítala na nových datech predikci.

V další části:
* Natrénujeme tento model pro $\lambda = 1$ (kvůli tomu, že má `lambda` speciální význam v Pythonu, tak parametr označíme alpha) a vypište jednotlivé koeficienty a intercept.
* Nakreslíme scatter plot hodnot $Y_i$ a $\hat Y_i$ pro validační množinu.
* Pro validační a testovací data provedeme predikce $\hat Y_i$ a porovnáme je se skutečnými hodnotami $Y_i$.
Jako míru porovnání použijeme RMSE.
* Porovnáme koeficienty získané lineární regresí a hřebenovou regresí. Provnáme výkonnosti obou modelů.

In [ ]:
# Váš kód pro přípravu třídy zde

class my_ridge:
    """
    Třída reprezentující model hřebenové regrese s rozhraním shodným jako mají sklearn modely
    """
    def __init__(self, alpha):
        """
        V konstruktoru nastavíme hyperparametr lambda označený zde alpha
        """
        self.alpha = alpha
        self.w_hat = None
    #------------------------------------
    def fit(self, Xtrain, ytrain):
        """
        Trénování spočívá v sestrojení w_hat
        """
       
        self.w_hat = ...
    #------------------------------------
    def predict(self, Xdata):
        """
        Predikce pro daná data
        """

        return ...
    #------------------------------------
    @property
    def intercept_(self):
        """
        Intercept
        """
        return self.w_hat[0] if self.w_hat is not None else None
    #------------------------------------
    @property
    def coef_(self):
        """
        Remaining fitted parameters
        """
        return self.w_hat[1:] if self.w_hat is not None else None

In [ ]:
# Využijeme implementovanou třídu

clf_my_ridge = my_ridge(1)
clf_my_ridge.fit(Xtrain, ytrain)

print(f"w_hat_my_ridge: {clf_my_ridge.intercept_:.4f}", clf_my_ridge.coef_)

RMSE_val_my_ridge = mean_squared_error(yval, clf_my_ridge.predict(Xval), squared = False)
print(f'Validační RMSE: {RMSE_val_my_ridge:.5f}')

RMSE_test_my_ridge = mean_squared_error(ytest, clf_my_ridge.predict(Xtest), squared = False)
print(f'Testovací RMSE: {RMSE_test_my_ridge:.5f}')

plt.scatter(yval, clf_my_ridge.predict(Xval))
plt.plot([0,50], [0,50], 'r')
plt.show()

plt.plot(clf_my_ridge.coef_, 'k.', label="Ridge regression")
plt.plot(clf_lr.coef_, 'rx', label="Linear regression")
plt.legend()
plt.show()

## Úkol - model hrebeňovej regresie s využitím scikit-learn

* Zopakujte postup z předchozího bodu s využitím třídy [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) ze `scikit-learn`.
* Porovnejte naučené koeficienty a výkonnost modelu s ruční implementací.

In [ ]:
# Váš kód zde



Vidíme, že jsme dostali přesně to samé, jako ruční implementací.

## Úkol - ladění hyperparametrů hřebenové regrese

* Nejprve pro několik různých hodnot $\lambda > 0$ vykreslete validační RMSE a odhadněte vhodný interval, ve kterém se bude nacházet optimální hodnota.
* Využijte funkci `scipy.optimize.minimize_scalar` k nalezení optimální hodnoty.
* Pro optimální hodnotu $\lambda$ natrénujte nový model.
* Výsledné RMSE pro testovací data porovnejte s předchozími výsledky.

In [ ]:
from scipy import optimize

# Váš kód zde



Vidíme, že jsme dostali pouze nepatrné vylepšení a klasická lineární regrese je zde stále lepší.

## Úkol - zopakujte předchozí úlohu pro standardizované příznaky
* Ke standardizaci využijte `sklearn.preprocessing.StandardScaler`.
* Výsledné RMSE pro testovací data porovnejte s výstupem lineární regrese a předchozí hřebenové regrese.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Váš kód zde



Vidíme, že teprve normalizace přinesla významné zlepšení modelu, který je nyní přesnější než původní lineární regrese.

## Vztah vychýlení a rozptylu
Na umělých datech numericky prozkoumáme vztah vychýlení a rozptylu.

<center><img src="img/tradeoff.jpeg" width="50%"></center>
<center>(zdroj: https://medium.com/@mp32445/understanding-bias-variance-tradeoff-ca59a22e2a83)</center>


Nejprve si připravíme funkce na náhodný výběr z modelu

In [ ]:
np.random.seed(3)  # zajistí replikovatelnost

# body x, kde budou data
x = np.random.rand(50)*5 - 1

# body x pro vykreslování
x_plot = np.linspace(-1,4,300)

# funkce, která vytvoří dataset - příznaky, jako mocniny x
def getX(x, max_degree = 5):
    X = x.reshape(-1,1)
    for i in range(2, max_degree+1):
        X = np.concatenate((X, (x**i).reshape(-1,1)), axis = 1)
    return X

# dataset pro X
X = getX(x)

# dataset pro vykreslení
X_plot = getX(x_plot)
print('X shape:', X.shape)

# příprava neznámého vektoru parametrů
true_intercept = 1
true_coefs = [-1,2.5,0,-0.11,0]

# funkce, která vrací náhodný výběr ze známého modelu
def getY(X, random = True, true_intercept = true_intercept, true_coefs = true_coefs):
    
    # vytvoření skutečného w
    w = np.concatenate((np.array([true_intercept] + true_coefs), np.zeros(1000)),)
    w = w[0:(X.shape[1]+1)]
    # přidání interceptu k matici příznaků
    Xx = np.concatenate((np.ones(X.shape[0]).reshape(-1,1),X), axis = 1)
    # nagenerujeme z modelu trénovací množiny střední hodnoty v bodech x
    EY = Xx.dot(w)
    # pokud je zapnutá náhodná odchylka, přidáme ji
    if random:
        # reset random seedu, aby to bylo náhodné
        np.random.seed()
        # přidání náhodné odchylky
        return EY + np.random.randn(X.shape[0])*1
    else:
        return EY

# vezmeme si jeden trénovací dataset
Y = getY(X)
# zobrazíme ho
plt.scatter(x,Y)

# vyrobíme si skutečné střední hodnoty
EY_plot = getY(X_plot, random = False)
plt.plot(x_plot, EY_plot, 'r')
plt.show()

### Nyní prozkoumáme vychýlení a rozptyl odhadu lineární a hřebenové regrese
Zafixujeme $\lambda = 0$ (alpha) a 100x provedeme:
* Nagenerování trénovacích hodnot $Y$.
* Predikci pomocí hřebenové regrese pro testovací hodnoty `X_plot`.

Výslednou predikci, tj. křivku dvojic `(x_plot, Yhat_plot)`, vykreslíme do jednoho grafu spolu se výběrovým průměrem přes všechny predikce.

Podíváme se na chování odhadu $\hat w$ (střední hodnotu).

Podíváme se, jak se výsledky mění v závislosti se zvyšujícím $\lambda$.

In [ ]:
def ridge_repeat(alpha = 1, N = 100, plot = True):
    Yres = np.zeros((N, X_plot.shape[0]))
    intercept_res = np.zeros((N, ))
    coef_res = np.zeros((N, X_plot.shape[1]))
    
    clf = Ridge(alpha = alpha)
    for i in range(N):
        clf.fit(X, getY(X))
        Yth_plot = clf.predict(X_plot)
        Yres[i,:] = Yth_plot
        intercept_res[i] = clf.intercept_
        coef_res[i,:] = clf.coef_
        if plot:
            if N % 2 == 0:
                plt.plot(x_plot,Yth_plot, 'gray')
                
    plt.plot(x_plot, EY_plot, 'r')
    plt.plot(x_plot, np.mean(Yres, axis = 0))
    plt.show()
    return Yres, intercept_res, coef_res


Yres, intercept_res, coef_res  = ridge_repeat(alpha = 1, N = 200)


# print těch různých statistik
# what
E_intercept_ridge = np.mean(intercept_res, axis=0)
print(f"Mean intercept : {E_intercept_ridge}")
print(f"True intercept : {true_intercept}")
print("---------------------\n")

E_coefs_ridge = np.mean(coef_res, axis=0)
print(f"Mean coefs : {E_coefs_ridge}")
print(f"True coefs : {true_coefs}")
print("---------------------\n")

# Yhat
E_Y_hat_ridge = np.mean(Yres, axis=0)
bias_ridge = E_Y_hat_ridge - EY_plot
var_ridge = np.var(Yres, axis=0)
print("Mean Ridge Regression of squared Biases on test set : " + str(np.mean(np.square(bias_ridge))))
print("Mean Ridge Regression Variance on test set : " + str(np.mean(var_ridge)))

Vidíme, že pro $\lambda = 0$ je odhad nestranný. Pro restoucí $\lambda$ přestane být nestranný a začně klesat variance.

Podívejme se na to, že v různých bodech je různý bias i variance.

In [ ]:
plt.plot(x_plot,bias_ridge, 'blue')
plt.plot(x_plot,np.var(Yres, axis = 0),'orange')
plt.show()